In [10]:
import os.path
from typing import Iterable, Optional, Tuple, Union, cast, Any, List, Type
import json

import rdflib
from rdflib import Graph, Dataset, Literal
from rdflib.term import Node
from rdflib.namespace import RDF, URIRef, OWL

from rdflib import Namespace
import csv


In [2]:
# load usda trig
usda_dataset = rdflib.Dataset()
usda_dataset.parse("Dataset/usda.trig")

<Graph identifier=file:///Users/kondy/PycharmProjects/Understanding_Recipies/Dataset/usda.trig (<class 'rdflib.graph.Graph'>)>

In [3]:
# Find nutritional information

nanopublication_namespace = Namespace("http://www.nanopub.org/nschema#")
usda_kb_namespace = Namespace("http://idea.rpi.edu/heals/kb/usda#")

nanopublication_iri_set: set[Node] = set()


In [8]:
# each nanopublication refers to one assertion node, that contains the entry of all nutritional info of one ingredient
for nanopublication_iri, _, _, _ in usda_dataset.quads((None, RDF.type, nanopublication_namespace.term("Nanopublication"), None)):
    # get the assertion node that refers to the same ingredient
    print("Nanopublication IRI:", nanopublication_iri)
    break

Nanopublication IRI: http://idea.rpi.edu/heals/kb/usda#nanoPub-explicit_entry-358cf0c481bd295b4ce102e65ee4d18a


In [6]:
for _, _, nano_assertion_iri, _ in usda_dataset.quads((nanopublication_iri, nanopublication_namespace.term("hasAssertion"), nanopublication_namespace.term("Nanopublication"), None)):
    print("Nano Assertion IRI:", nano_assertion_iri)
    break

In [ ]:

# knows_query = """
# SELECT DISTINCT ?aname ?bname
# WHERE {
#     ?a foaf:knows ?b .
#     ?a foaf:name ?aname .
#     ?b foaf:name ?bname .
# }"""

# qres = g.query(knows_query)
# for row in qres:
#     print(f"{row.aname} knows {row.bname}")

In [ ]:
# read the usda FoodKG mappings:


In [34]:
# read the usda nutritional info

usda_id_to_name_dict = dict()

with open('Dataset/usda.csv', newline='') as csvfile:

    spamreader = csv.reader(csvfile, delimiter=',', quotechar='"')
    i = 0
    
#     read column names:

    for row in spamreader:
        headers_row = row
        print(headers_row)
        print(len(row))
        break
    
    for row in spamreader:
#         print(row)
        ingredient_id = int(row[0])
        ingredient_name = row[1]
        usda_id_to_name_dict[ingredient_id] = ingredient_name
        
#         assert len(row) == 43
        
#         print(len(row))
#         break
        
        
#         print(', '.join(row))
        i += 1
#         if i%1000 == 0:
#             print(i)
    print(len(usda_id_to_name_dict))

['\ufeffid', 'description', 'water', 'energy', 'protein', 'lipid', 'ash', 'carbohydrate', 'fiber', 'sugar', 'calcium', 'iron', 'magnesium', 'phosphorus', 'potassium', 'sodium', 'zinc', 'copper', 'manganese', 'selenium', 'vitamin_C', 'thiamin', 'riboflavin', 'niacin', 'panto_acid', 'vitamin_B6', 'food_folate', 'choline', 'vit_B12', 'vit_A', 'vit_A_RAE', 'retinol', 'alpha_carot', 'beta_carot', 'beta_crypt', 'lycopene', 'vitamin_E', 'vit_D', 'vit_K', 'fat_sat', 'fat_mono', 'fat_poly', 'cholestrl']
43
8618


In [36]:
# see how many USDA_IDs are in the usda foodkg mappings

# load the mappings
foodkg_to_usda_equivalent_classes_filename: str = "Dataset/foodkg_usda_eq_classes_graph.ttl"
foodkg_to_usda_equivalent_classes_graph: Graph = Graph()
foodkg_to_usda_equivalent_classes_graph.parse(foodkg_to_usda_equivalent_classes_filename)

# usda ID mappings matched to the usda_IRI
food_kg_IRI_to_usda_name_dict = dict()

print("Total mappings:")

unique_usda_IDs = set()

# for each foodkg - usda mapping
for foodkg_IRI, _, usda_ID_IRI in foodkg_to_usda_equivalent_classes_graph.triples((None, None, None)):
    usda_ID = int(str(usda_ID_IRI.split("#")[-1]))
    
    # extract the usda ID from the IRI
    if usda_ID in usda_id_to_name_dict:
        food_kg_IRI_to_usda_name_dict[foodkg_IRI] = usda_id_to_name_dict[usda_ID]
        unique_usda_IDs.add(usda_ID)
        
print("Matches found:", len(food_kg_IRI_to_usda_name_dict))
print("Over", len(unique_usda_IDs), "number of usda distinct entities.")


    
    


Total mappings:
Matches found: 15467
Over 1434 number of usda distinct entities.


In [19]:
# Find nutritional information

nanopublication_namespace = Namespace("http://www.nanopub.org/nschema#")
usda_kb_namespace = Namespace("http://idea.rpi.edu/heals/kb/usda#")

nanopublication_iri_set: set[Node] = set()

# each nanopublication refers to one assertion node, that contains the entry of all nutritional info of one ingredient
for nanopublication_iri, _, _, _ in usda_dataset.quads((None, RDF.type, nanopublication_namespace.term("Nanopublication"), None)):
    # get the assertion node that refers to the same ingredient
    print("Nanopublication IRI:", nanopublication_iri)
    break
for _, _, nano_assertion_iri, _ in usda_dataset.quads((nanopublication_iri, nanopublication_namespace.term("hasAssertion"), nanopublication_namespace.term("Nanopublication"), None)):
    print("Nano Assertion IRI:", nano_assertion_iri)
    break
    # get all nutritional info about this ingredient
for nutr_node_ID, _, _, _ in usda_dataset.quads((None, RDF.type, usda_kb_namespace.term("id"), nano_assertion_iri)):
    print("Nutrinitonal Info entry node ID:", nutr_node_ID)
    break

Nanopublication IRI: http://idea.rpi.edu/heals/kb/usda#nanoPub-explicit_entry-358cf0c481bd295b4ce102e65ee4d18a


NameError: name 'nano_assertion_iri' is not defined

In [10]:
# query uskb entries
usda_kb_id_uriref = URIRef("http://idea.rpi.edu/heals/kb/usda#id")

# relate uskb id nodes to their context node element
matches_found = 0
uskb_id_node_to_context_element_dict = dict()
for usda_kb_id_node, _, _, context_element in usda_dataset.quads((None, RDF.type, usda_kb_id_uriref, None)):
    matches_found += 1
    uskb_id_node_to_context_element_dict[usda_kb_id_node] = context_element

print("Total quad patterns matched:", matches_found)
print("Total USKB elements found:", len(uskb_id_node_to_context_element_dict))



Total quad patterns matched: 8618
Total USKB elements found: 8618


In [ ]:
# gather all nutritient information about each ingredient

nutritient_info_node = set()

context_element_set = list(uskb_id_node_to_context_element_dict.values())

for context_element_node in context_element_set:
    ingredient_info_entry_node = set()
    for nutritient_info_entry_node, _, _, context_element_node in usda_dataset.quads((None, None, None, context_element_node)):
